In [1]:
import os
import mysql.connector
import pandas as pd

# Fetches all data form the database regarding the charger geodata
def get_charger_geocoded():
    # Set your environment variables or provide them directly in the code
    host = os.environ.get("MYSQL_HOST", "localhost")
    port = os.environ.get("MYSQL_PORT", "3306")
    dbname = os.environ.get("MYSQL_DBNAME", "test")
    user = os.environ.get("MYSQL_USER", "root")
    password = os.environ.get("MYSQL_PASSWORD", "")

    try:
        conn = mysql.connector.connect(
            host=host,
            port=port,
            user=user,
            password=password,
            database=dbname
        )
        if conn.is_connected():
            # Create a cursor
            cursor = conn.cursor()
            
            # Define your SELECT query
            query = "SELECT * FROM charger_data_geocoded"
            
            # Execute the query
            cursor.execute(query)
            
            # Fetch the results into a Pandas DataFrame
            result = cursor.fetchall()
            result_df = pd.DataFrame(result, columns=cursor.column_names)

            return result_df
        else:
            print("Connection to MySQL database failed.")
    except mysql.connector.Error as err:
        print(f"Error: {err}")

# Fetches all data from the database regardign pop dens
def get_pop_dens():
    # Set your environment variables or provide them directly in the code
    host = os.environ.get("MYSQL_HOST", "localhost")
    port = os.environ.get("MYSQL_PORT", "3306")
    dbname = os.environ.get("MYSQL_DBNAME", "test")
    user = os.environ.get("MYSQL_USER", "root")
    password = os.environ.get("MYSQL_PASSWORD", "")

    try:
        conn = mysql.connector.connect(
            host=host,
            port=port,
            user=user,
            password=password,
            database=dbname
        )
        if conn.is_connected():
            # Create a cursor
            cursor = conn.cursor()
            
            # Define your SELECT query
            query = "SELECT * FROM pop_dens"
            
            # Execute the query
            cursor.execute(query)
            
            # Fetch the results into a Pandas DataFrame
            result = cursor.fetchall()
            result_df = pd.DataFrame(result, columns=cursor.column_names)

            return result_df
        else:
            print("Connection to MySQL database failed.")
    except mysql.connector.Error as err:
        print(f"Error: {err}")


In [2]:
import geopandas as gpd

# Load the GeoJSON file into a GeoDataFrame
cantons_gdf = gpd.read_file('UsedData/cantons.geojson')

# Fetch charger data from database (country / lat / lon)
df_clean = get_charger_geocoded()

# Assuming your dataframe with longitude and latitude is named df_clean
# Convert df_clean into a GeoDataFrame
geometry = gpd.points_from_xy(df_clean.lon, df_clean.lat)
stations_gdf = gpd.GeoDataFrame(df_clean, geometry=geometry)

# Perform spatial join
joined_gdf = gpd.sjoin(stations_gdf, cantons_gdf, op="within")

# Identify records in stations_gdf that didn't join with cantons_gdf
unmatched_chargers = stations_gdf[~stations_gdf.index.isin(joined_gdf.index)]

# Add the canton abbreviation to df_clean
df_clean['canton_short'] = joined_gdf['id']
df_clean['canton'] = joined_gdf['name']
df_clean = df_clean.dropna()
# If you want to reset the index for unmatched_stations for better readability
# unmatched_stations = unmatched_stations.reset_index(drop=True)



canton_mapping = {
    'Bern/Berne': 'Bern',
    'Genève': 'Genf',
    'Valais/Wallis':'Wallis',
    'Graubünden/Grigioni':'Graubünden',
    'Ticino':'Tessin',
    'Vaud':'Waadt',
    'Neuchâtel':'Neuenburg'
    }


df_clean['canton'] = df_clean['canton'].replace(canton_mapping)

df_clean.to_csv('Results/charger_data_geocoded_new.csv', index=False)

print(df_clean)
print(unmatched_chargers)

      country        lat       lon canton_short      canton
0         CHE  47.418408  8.373644           AG      Aargau
1         CHE  47.028107  8.631626           SZ      Schwyz
2         CHE  47.418408  8.373644           AG      Aargau
3         CHE  47.028107  8.631626           SZ      Schwyz
4         CHE  47.028107  8.631626           SZ      Schwyz
...       ...        ...       ...          ...         ...
12495     CHE  47.519055  8.718913           ZH      Zürich
12496     CHE  47.355510  8.325744           AG      Aargau
12497     CHE  47.479983  9.531401           SG  St. Gallen
12498     CHE  47.355510  8.325744           AG      Aargau
12499     CHE  47.355510  8.325744           AG      Aargau

[12490 rows x 5 columns]
     country        lat       lon                  geometry
2832     CHE  47.152033  9.511593  POINT (9.51159 47.15203)
2833     CHE  47.152033  9.511593  POINT (9.51159 47.15203)
2834     CHE  47.151997  9.511668  POINT (9.51167 47.15200)
2836     CHE  

/Users/matis/Library/Python/3.10/lib/python/site-packages/IPython/core/interactiveshell.py:3466: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [3]:
df = get_pop_dens()
df_inhabitant = df[['GEO_NAME', 'UNIT', 'VALUE']]
df_inhabitant = df_inhabitant[df_inhabitant['UNIT'] == "Einwohner/Innen"]
df_popDens = df[['GEO_NAME', 'UNIT', 'VALUE']]
df_popDens = df_popDens[df_popDens['UNIT'] != "Einwohner/Innen"]

#adding Freiburg, bc is was missing

# Create a new DataFrame for the new row
new_row_popDens = pd.DataFrame([{'GEO_NAME':'Freiburg', 'UNIT':'Einwohner pro km²', 'VALUE':210.1}])
new_row_inhabitants = pd.DataFrame([{'GEO_NAME':'Freiburg', 'UNIT':'Einwohner/Innen', 'VALUE':334465}])

# Concatenate df_popDens and the new row
df_popDens = pd.concat([df_popDens, new_row_popDens], ignore_index=True)
df_inhabitants = pd.concat([df_inhabitant, new_row_inhabitants], ignore_index=True)

df_popDens.rename(columns={'VALUE':'popDens', 'GEO_NAME': 'canton'}, inplace=True)
df_inhabitant.rename(columns={'VALUE':'inhabitants', 'GEO_NAME': 'canton'}, inplace=True)

In [5]:
import requests
from bs4 import BeautifulSoup
import re

# List of canton abbreviations
cantons = {

    'Aargau': 'ag',
    'Appenzell Innerrhoden': 'ai',
    'Appenzell Ausserrhoden': 'ar',
    'Basel-Stadt': 'bs',
    'Basel-Landschaft': 'bl',
    'Bern': 'be',
    'Freiburg': 'fr',
    'Genf': 'ge',
    'Glarus': 'gl',
    'Graubünden': 'gr',
    'Jura': 'ju',
    'Luzern': 'lu',
    'Neuenburg': 'ne',
    'Nidwalden': 'nw',
    'Obwalden': 'ow',
    'Schaffhausen': 'sh',
    'Schwyz': 'sz',
    'Solothurn': 'so',
    'St. Gallen': 'sg',
    'Thurgau': 'tg',
    'Tessin': 'ti',
    'Uri': 'ur',
    'Wallis': 'vs',
    'Waadt': 'vd',
    'Zug': 'zg',
    'Zürich': 'zh'
}

#crating lists to store information, to later put into a dataframe
mean_incomes_list = []
canton_list = []
abbreviation_list = []


# Dictionary to store median incomes
median_incomes = {}

pattern = re.compile(r'CHF ([\d ]+)')

# Loop through each canton and scrape the median income
for canton, abbreviation in cantons.items():
    url = f"https://www.jobs.ch/de/lohn/kanton/?canton={abbreviation}"
    response = requests.get(url)

    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract all the <p> tags from the body
    paragraphs = soup.body.find_all('p')

    #print(paragraphs[23].get_text())

    text =paragraphs[23].get_text()

	#regex to get median income numercial value
    matches = re.findall(r"([\d ]+)", text)
    mean_income = int(matches[0].replace('\u202f', ''))
    #print(mean_income)


    # Append the values to the lists
    mean_incomes_list.append(mean_income)
    canton_list.append(canton)
    abbreviation_list.append(abbreviation.upper())

# Create a DataFrame
scrape_df = pd.DataFrame({
    'medianIncome': mean_incomes_list,
    'canton': canton_list,
    'abbreviation': abbreviation_list
})

scrape_df = scrape_df[['canton', 'medianIncome', 'abbreviation']]

print(df_inhabitant.dtypes)
print(df_popDens.dtypes)
print(scrape_df.dtypes)
scrape_df

canton          object
UNIT            object
inhabitants    float64
dtype: object
canton      object
UNIT        object
popDens    float64
dtype: object
canton          object
medianIncome     int64
abbreviation    object
dtype: object


,canton,medianIncome,abbreviation
0,Aargau,75391,AG
1,Appenzell Innerrhoden,84535,AI
2,Appenzell Ausserrhoden,71911,AR
3,Basel-Stadt,78200,BS
4,Basel-Landschaft,77061,BL
5,Bern,74886,BE
6,Freiburg,71996,FR
7,Genf,77001,GE
8,Glarus,72860,GL
9,Graubünden,70130,GR


In [6]:
df_inhabitant = df_inhabitant.drop_duplicates(subset='canton')
scrape_df = scrape_df.drop_duplicates(subset='canton')
df_popDens = df_popDens.drop_duplicates(subset='canton')

print(df_inhabitant)
print(scrape_df)
print(df_popDens)


final_scraped_df = pd.merge(df_inhabitant, scrape_df, on='canton')
final_df = pd.merge(df_popDens, final_scraped_df, on='canton')
canton_data_df = final_df[['canton', 'abbreviation', 'popDens', 'medianIncome', 'inhabitants']]

                    canton             UNIT  inhabitants
0                   Zürich  Einwohner/Innen    1579967.0
2                     Bern  Einwohner/Innen    1051437.0
4                   Luzern  Einwohner/Innen     424851.0
6                      Uri  Einwohner/Innen      37317.0
8                   Schwyz  Einwohner/Innen     164920.0
10                Obwalden  Einwohner/Innen      38700.0
12               Nidwalden  Einwohner/Innen      44420.0
14                  Glarus  Einwohner/Innen      41471.0
16                     Zug  Einwohner/Innen     131164.0
18                Freiburg  Einwohner/Innen     334465.0
20               Solothurn  Einwohner/Innen     282408.0
22             Basel-Stadt  Einwohner/Innen     196786.0
24        Basel-Landschaft  Einwohner/Innen     294417.0
26            Schaffhausen  Einwohner/Innen      85214.0
28  Appenzell Ausserrhoden  Einwohner/Innen      55759.0
30   Appenzell Innerrhoden  Einwohner/Innen      16416.0
32              St. Gallen  Ein

## We want to categorice the cantons in to rural and urban. This way we can control our data better when we do our calculations

In [7]:
# Add new column with category "urban" or "rural" depending on population density
canton_data_df['rural'] = 1
canton_data_df.loc[canton_data_df['popDens'] > 900, 'rural'] = 0
canton_data_df

/var/folders/v1/5hsb0rrj4wjfjsbnzc38bkb80000gn/T/ipykernel_20739/3840127681.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  canton_data_df['rural'] = 1


,canton,abbreviation,popDens,medianIncome,inhabitants,rural
0,Zürich,ZH,951.3,83270,1579967.0,0
1,Bern,BE,180.1,74886,1051437.0,1
2,Luzern,LU,297.3,73678,424851.0,1
3,Uri,UR,35.3,71500,37317.0,1
4,Schwyz,SZ,193.7,75020,164920.0,1
5,Obwalden,OW,80.5,72123,38700.0,1
6,Nidwalden,NW,184.0,75978,44420.0,1
7,Glarus,GL,60.9,72860,41471.0,1
8,Zug,ZG,633.2,84995,131164.0,1
9,Freiburg,FR,210.1,71996,334465.0,1


## Please do only use tne following code once. Else the data will be added double to the database!

In [8]:
import os
import mysql.connector

def save_canton_data_in_db(df):
    # Set your environment variables or provide them directly in the code
    host = os.environ.get("MYSQL_HOST", "localhost")
    port = os.environ.get("MYSQL_PORT", "3306")
    dbname = os.environ.get("MYSQL_DBNAME", "test")
    user = os.environ.get("MYSQL_USER", "root")
    password = os.environ.get("MYSQL_PASSWORD", "")

    table_name = "canton_data"

    print(df)

    try:
        conn = mysql.connector.connect(
            host=host,
            port=port,
            user=user,
            password=password,
            database=dbname
        )
        if conn.is_connected():
            # Create a cursor
            cursor = conn.cursor()

            print(df)

            for index ,row in df.iterrows():
                query = f"INSERT INTO {table_name} (canton, abbreviation, popDens, medianIncome, inhabitants, rural) VALUES (%s, %s, %s, %s, %s, %s)"
                values = (row['canton'], row['abbreviation'], row['popDens'], row['medianIncome'], row['inhabitants'], row['rural'])
                cursor.execute(query, values)

            conn.commit()
            cursor.close()
        else:
            print("Connection to MySQL database failed.")
    except mysql.connector.Error as err:
        print(f"Error: {err}")

    finally: 
        if conn is not None:
            conn.close()


In [9]:
# Save the data to the Database
save_canton_data_in_db(canton_data_df)

                    canton abbreviation  popDens  medianIncome  inhabitants  \
0                   Zürich           ZH    951.3         83270    1579967.0   
1                     Bern           BE    180.1         74886    1051437.0   
2                   Luzern           LU    297.3         73678     424851.0   
3                      Uri           UR     35.3         71500      37317.0   
4                   Schwyz           SZ    193.7         75020     164920.0   
5                 Obwalden           OW     80.5         72123      38700.0   
6                Nidwalden           NW    184.0         75978      44420.0   
7                   Glarus           GL     60.9         72860      41471.0   
8                      Zug           ZG    633.2         84995     131164.0   
9                 Freiburg           FR    210.1         71996     334465.0   
10               Solothurn           SO    357.3         74505     282408.0   
11             Basel-Stadt           BS   5325.7    